In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.abspath('../src/'))
sys.path.append(os.path.abspath('../'))

In [2]:
from src.classifier import Classifier
from src.utils import cal_metric, print_results
from src.dataset import CANDataset
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import numpy as np

/Users/htnu/opt/miniconda3/envs/torch/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
data_dir = '../../Data/LISA/Federated_Data/Preprocessed_Data/Tesla/1/'
ckpt_files = '../save/lightning_logs/version_1/checkpoints/epoch=49-step=54400.ckpt'
model = Classifier.load_from_checkpoint(ckpt_files, num_classes=3, data_dir=data_dir)
transform = None
test_dataset = CANDataset(root_dir=data_dir, is_train=False, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, 
                        pin_memory=True, sampler=None)
trainer = pl.Trainer(enable_checkpointing=False, logger=False)
results = trainer.predict(model, dataloaders=test_loader)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

In [7]:
labels = np.concatenate([x['labels'] for x in results])
preds = np.concatenate([x['preds'] for x in results])
cm, metrics = cal_metric(labels, preds)

In [8]:
classes = ['Normal', 'Fuzzy', 'Replay']
print_results(metrics, classes)

	fnr	rec	pre	f1
Normal	0.076	0.9992	0.9976	0.9984
Fuzzy	0.0	1.0	1.0	1.0
Replay	6.3369	0.9366	0.9791	0.9574
